<a href="https://colab.research.google.com/github/felkira/stale_food_detection/blob/main/stale_food_detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import tensorflow as tf
import tensorflow.keras.backend as K
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.inception_v3 import preprocess_input
from tensorflow.keras.applications.inception_v3 import InceptionV3
from tensorflow.keras import regularizers
from tensorflow.keras.regularizers import l2
from keras_preprocessing.image import ImageDataGenerator
import pandas as pd
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.image as img
%matplotlib inline

In [ ]:
food_list = ['apel_segar', 'jeruk_segar', 'pisang_segar', 'tomat_segar', 'apel_busuk', 'jeruk_busuk', 'pisang_busuk', 'tomat_busuk']

In [ ]:
def plot_accuracy(history, title):
    plt.title(title)
    plt.plot(history.history['acc'])
    plt.plot(history.history['val_acc'])
    plt.ylabel('accuracy')
    plt.xlabel('epoch')
    plt.legend(['train_accuracy', 'validation_accuracy'], loc='best')
    plt.show()


def plot_loss(history, title):
    plt.title(title)
    plt.plot(history.history['loss'])
    plt.plot(history.history['val_loss'])
    plt.ylabel('loss')
    plt.xlabel('epoch')
    plt.legend(['train_loss', 'validation_loss'], loc='best')
    plt.show()

In [ ]:
def recall(y_true, y_pred):
     true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
     possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
     recall = true_positives / (possible_positives + K.epsilon())
     return recall

def precision(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + K.epsilon())
    return precision

In [ ]:
def model_train():
    DIR = '/content/drive/MyDrive/bangkits-fruit'

    train_datagen = ImageDataGenerator(preprocessing_function = preprocess_input,
                                       validation_split=0.25,
                                       shear_range = 0.2,
                                       zoom_range = 0.2,
                                       horizontal_flip = True)
    
    test_datagen = ImageDataGenerator(preprocessing_function = preprocess_input,  validation_split=0.25)
    
    train_generator = train_datagen.flow_from_directory(DIR,
                                                        target_size = (299, 299),
                                                        shuffle=True,
                                                        subset='training',
                                                        class_mode = 'categorical')
    
    validation_generator = test_datagen.flow_from_directory(DIR,
                                                            target_size = (299, 299),
                                                            shuffle=True,
                                                            subset='validation',
                                                            class_mode = 'categorical')
    
    inception = InceptionV3(weights = 'imagenet', include_top = False)
    x = inception.output
    x = tf.keras.layers.GlobalAveragePooling2D()(x)
    x = tf.keras.layers.Dense(512, activation = 'relu')(x)
    x = tf.keras.layers.Dropout(0.2)(x)

    predictions = tf.keras.layers.Dense(8, kernel_regularizer = regularizers.l2(0.005), activation = 'softmax')(x)

    model = tf.keras.models.Model(inputs = inception.input, outputs = predictions)
    opt = tf.keras.optimizers.Nadam(learning_rate = 1e-5, beta_1 = 0.9, beta_2 = 0.999, epsilon = 1e-07)
    model.compile(optimizer = opt, loss = 'categorical_crossentropy', metrics = ['acc', recall, precision])
    model.summary()

    class myCallback(tf.keras.callbacks.Callback):
        def on_epoch_end(self, epoch, logs={}):
            if (logs.get('acc') > 0.91 and logs.get('val_acc') > 0.91):
                print("\nReach the desired accuracy and validation_accuracy!")
                self.model.stop_training = True

    callbacks = myCallback()
    history = model.fit(train_generator,
                        steps_per_epoch = 50,
                        validation_data = validation_generator,
                        validation_steps = 10,
                        epochs = 10,
                        verbose = 1,
                        callbacks = [callbacks])
    
    print("\n")
    pred = model.predict(validation_generator)
    y_pred = np.argmax(pred, axis=1)
    y_true = validation_generator.classes
    con_mat = tf.math.confusion_matrix(labels=y_true, predictions=y_pred).numpy()
    con_mat_norm = np.around(con_mat.astype('float') / con_mat.sum(axis=1)[:, np.newaxis], decimals=2)
    classes = list(validation_generator.class_indices.keys())
    con_mat_df = pd.DataFrame(con_mat_norm, index = classes, columns = classes)
    figure = plt.figure(figsize=(10, 8))
    sns.heatmap(con_mat_df, annot=True, cmap=plt.cm.Blues)
    plt.tight_layout()
    plt.ylabel('True label')
    plt.xlabel('Predicted label')
    plt.show()
    print("\n")

    score = model.evaluate(validation_generator, verbose=0)
    print("Test Accuracy\t: ", score[1])
    print("Test Recall\t: ", score[2])
    print("Test Precision\t: ", score[3])
    f1score = 2*score[2]*score[3]/(score[2] + score[3])
    print("Test F1-Score\t: ", f1score)
    print("\n")

    plot_accuracy(history, 'bangkits-fruit_inceptionV3')
    plot_loss(history, 'bangkits-fruit_inceptionV3')
    return model

In [ ]:
model = model_train()
model.save("model.h5")

Found 5880 images belonging to 8 classes.
Found 1960 images belonging to 8 classes.
87924736/87910968 [==============================] - 1s 0us/step
Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, None, None,  0           []                               
                                 3)]                                                              
                                                                                                  
 conv2d (Conv2D)                (None, None, None,   864         ['input_1[0][0]']                
                                32)                                                               
                                                                                                  
 batch_normalization (BatchNorm  (None, None

In [ ]:
optimization = tf.lite.Optimize.DEFAULT

converter = tf.lite.TFLiteConverter.from_keras_model(model)
converter.optimizations = [optimization]
tflite_model = converter.convert()

with open('model.tflite', 'wb') as f:
  f.write(tflite_model)

In [ ]:
%%time
K.clear_session()
model_fix = load_model('model.h5', compile = False)

In [ ]:
def predict_class(model, images, show = True):
  for img in images:
    img = image.load_img(img, target_size=(299, 299))
    img = image.img_to_array(img)                    
    img = np.expand_dims(img, axis=0)         
    img = preprocess_input(img)                                      

    pred = model.predict(img)
    index = np.argmax(pred)
    food_list.sort()
    pred_value = food_list[index]
    #print(pred)
    if show:
        plt.imshow(img[0])                           
        plt.axis('off')
        plt.title(pred_value)
        plt.show()

In [ ]:
images = []
imagepath = '/content/drive/MyDrive/test_fruit/'
images.append(imagepath+'01.png')
images.append(imagepath+'02.png')
images.append(imagepath+'03.png')
images.append(imagepath+'04.png')
images.append(imagepath+'05.png')
images.append(imagepath+'06.png')
images.append(imagepath+'07.jpg')
images.append(imagepath+'08.png')
images.append(imagepath+'09.png')
images.append(imagepath+'10.png')
images.append(imagepath+'11.png')
images.append(imagepath+'12.png')
images.append(imagepath+'13.png')
images.append(imagepath+'14.jpg')
images.append(imagepath+'15.jpg')
predict_class(model_fix, images, True)